
Please __read the document carefuly__ and submit your code accordingly.  

Once you fill in the functions as explained below, disconnect runtime, reconnect and run all (i.e. ```Runtime -> Run all```) in order to double check that it still works. Delete any test code you have so that I do just see the functions below, I will write my own test code, clear all outuput (i.e. ```Edit -> Clear all outputs```), finally save this file as "**W4_studentID.ipynb**" and submit via [ODTU class](https://odtuclass2024f.metu.edu.tr/mod/assign/view.php?id=68205).  

Use of AI tools (such as the built-in Gemini in colab, or anyother you like) is allowed. However, if you use an AI tool, add the prompt(s) you used as a comment to the beginning of each code cell.  
Along with the comment, explain if the first prompt worked, if not explain how you fixed it, add all versions of your prompts in to your comments.


Allowed imports:  
- ```numpy```.  

Any submission:  
- with test code,  
- that crashes,  
- any other import than mentioned above
- not properly named

will not be graded.


## Full name:

## Student ID

## Definition:
You are to complete following functions. Details are explained in the docstring of each funciton.  

In the text cell before each function, in brief but sufficient detail, explain how the function calculates the desired outputs.

Note that these functions complement each other, you can test one with the other if they are properly written.  

Also note that, there is no limit for the dimension of ambient space.  

For the time being, there is no noise in data, i.e. rank from a particular subspace directly indicate the dimension of that subspace.  


### Generation of subspaces with desired guaranteed minimal or maximal angles

Replace the content of this cell where you briefly but sufficiently explain how you make sure that your code works using mathematical terms as much as possible.  

Note that you can write math type expression between $ $.  Recall numpy tutorial on ODTU Class.  

Anyway here are some examples just in case:  
$e = Mc^2$   

$\int sin(t) dt = -cos(t)$   

$\begin{bmatrix} 1 & 3 \\ 2 & 4 \end{bmatrix} \mathbf{x} = \begin{bmatrix} 4 \\ 6 \end{bmatrix}$

$\tilde{x} = 0.999 x$

In [381]:
import numpy as np


In [382]:
def GenerateSubspacesMin(D=5, dS1=2, dS2=2, nS1=50, nS2=50, minimalAngle=0):
    '''
    This function generates 2 data matrices in subspaces S1 and S2 respectively that live in D dimensional ambient space
    dSi is the dimension of subspace Si
    nSi is the number of data points to be generated in subspace Si
    minimalAngle is the minimal angle between subspaces, recall the definition in lecture notes
    Function returns two numpy arrays: M1, M2
    where:
        dimension of matrix Mi is (D, nSi)
        rank(Mi) = dSi
    if passed data does not make sense, return two empty numpy arrays
    '''
    try:
        # Type checking
        if not all(isinstance(x, (int, np.integer)) for x in [D, dS1, dS2, nS1, nS2]):
            return np.array([]), np.array([])
        if not isinstance(minimalAngle, (int, float, np.integer, np.floating)):
            return np.array([]), np.array([])
            
        # Value validation
        if not (0 <= minimalAngle <= np.pi/2):
            return np.array([]), np.array([])
        if not all(x > 0 for x in [D, dS1, dS2, nS1, nS2]):
            return np.array([]), np.array([])
        if not (dS1 <= D and dS2 <= D):
            return np.array([]), np.array([])
            
        # Check geometric possibility
        if D < dS1 + dS2 and minimalAngle > 0:
            return np.array([]), np.array([])

        # Generate random basis for first subspace
        B1 = np.random.randn(D, dS1)
        # Orthonormalize using QR decomposition
        Q1, R1 = np.linalg.qr(B1)
        B1 = Q1[:, :dS1]

        # Generate random basis for second subspace
        max_attempts = 1000
        attempt = 0
        success = False
        
        while attempt < max_attempts:
            try:
                B2 = np.random.randn(D, dS2)
                Q2, R2 = np.linalg.qr(B2)
                B2 = Q2[:, :dS2]
                
                # Check principal angles between subspaces
                U1, S, V1 = np.linalg.svd(B1.T @ B2, full_matrices=False)
                angles = np.arccos(np.clip(S, -1.0, 1.0))
                min_angle = np.min(angles)
                
                if min_angle >= minimalAngle:
                    success = True
                    break
                    
            except np.linalg.LinAlgError:
                pass
                
            attempt += 1
        
        if not success:
            return np.array([]), np.array([])

        # Generate random coefficients for points in each subspace
        C1 = np.random.randn(dS1, nS1)
        C2 = np.random.randn(dS2, nS2)

        # Generate points in each subspace
        M1 = B1 @ C1  # D x nS1 matrix
        M2 = B2 @ C2  # D x nS2 matrix

        return M1, M2
        
    except Exception:
        return np.array([]), np.array([])

### Finding minimal and maximal angles between subspaces  
Similar to the case above, replace the content of this cell to explain briefly but sufficiently how you make sure that your code works using mathematical terms as much as possible.  


In [383]:

def FindMinimalAngles(M1, M2):
    '''
    This function calculates and returns the minimal angle between subspaces S1, and S2
    that contain the data points M1 and M2 respectively.
    The minimal angle between subspaces is defined as the smallest principal angle
    between the subspaces spanned by the columns of M1 and M2.
    
    Parameters:
    M1, M2: numpy arrays representing data matrices
    
    Returns:
    float: minimal angle in radians between the subspaces
    -1000: if input data is invalid
    '''
    try:
        # Input validation
        if not isinstance(M1, np.ndarray) or not isinstance(M2, np.ndarray):
            return -1000
        
        # Check if matrices are empty
        if M1.size == 0 or M2.size == 0:
            return -1000
        
        # Check if matrices have same number of rows (same ambient dimension)
        if M1.shape[0] != M2.shape[0]:
            return -1000
        
        # Check if matrices are numeric
        if not (np.issubdtype(M1.dtype, np.number) and np.issubdtype(M2.dtype, np.number)):
            return -1000
            
        # Check for NaN or Inf values
        if np.any(np.isnan(M1)) or np.any(np.isnan(M2)) or \
           np.any(np.isinf(M1)) or np.any(np.isinf(M2)):
            return -1000

        # Find orthonormal basis for M1 using QR decomposition
        try:
            Q1, R1 = np.linalg.qr(M1)
            # Determine rank using SVD
            _, S1, _ = np.linalg.svd(M1, full_matrices=False)
            rank1 = np.sum(S1 > 1e-10)
            if rank1 == 0:
                return -1000
            Q1 = Q1[:, :rank1]

            # Find orthonormal basis for M2
            Q2, R2 = np.linalg.qr(M2)
            _, S2, _ = np.linalg.svd(M2, full_matrices=False)
            rank2 = np.sum(S2 > 1e-10)
            if rank2 == 0:
                return -1000
            Q2 = Q2[:, :rank2]
            
            # Compute SVD of Q1^T @ Q2
            U, S, Vh = np.linalg.svd(Q1.T @ Q2, full_matrices=False)
            
            # Compute the principal angles from the singular values
            angles = np.arccos(np.clip(S, -1.0, 1.0))
            
            # Return the smallest angle
            return float(np.min(angles))
            
        except np.linalg.LinAlgError:
            return -1000
            
    except Exception:
        return -1000

In [384]:
# All these cases will run without crashing
test_cases = [
    # Normal case
    (4, 2, 2, 50, 50, np.pi/4),
    # Impossible geometry
    #(3, 2, 2, 50, 50, np.pi/4),
    # Bad inputs
    #(-1, 2, 2, 50, 50, 0),
    #('a', 2, 2, 50, 50, 0),
    #(4, 5, 2, 50, 50, 0),
    #(4, 2, 2, 50, 50, 2*np.pi)
]

for args in test_cases:
    M1, M2 = GenerateSubspacesMin(*args)
    angle = np.degrees(FindMinimalAngles(M1, M2))
    # Will either get valid results or appropriate error values

In [385]:
print(angle)

47.534391445596206
